# Softmax Classification

- softmax
- cross entropy
- low-level implementation
- high-level implementation
- training example

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

## Discrete Probability Distribution (이산확률분포)

- 이산확률분포란, 확률분포함수가 이어지지 않는 방식으로 나온다
- 즉 a, c, e라는 x가 존재할 때 a와 c 사이의 b에 대한 확률이 존재하지 않게 된다. 
- 이런 이산확률분포에서 특정 값에 대한 예측을 진행하는 것이 softmax 함수이다. 

In [8]:
z = torch.FloatTensor([1, 2, 3])

hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [9]:
hypothesis.sum()

tensor(1.)

## Cross Entropy Loss(Low Level)

- 두 분포 사이의 차이를 측정하는데 활용한다. 
- classification에서 모델이 예측한 확률 분포와 실제 확률 분포의 차이를 확인할 때 사용한다. 
- CE(p, q) = - Σ y * log(Y) (y = 실제 정답, Y = 모델이 예측한 확률) = - Σ p(x) log( q(x) )
- 정답에 가까울 수록 loss가 작아진다

In [10]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [11]:
y = torch.randint(5, (3, )).long()
print(y)

tensor([0, 2, 1])


## Cross Entropy Loss(Low-level)

discrete한 확률분포기 때문에 one-hot 벡터로 표현할 수 있다. 

> 왜 discrete한데 one-hot 벡터로 표현할까? 
> one-hot vector : 오직 하나의 차원만 1로 표현된 벡터(나머지는 0이다)
> 이산확률분포(discrete) : 각 값이 a, b, c, d중 하나에 해당된다. 이를 차원으로 만들어 (an, bn, cn, dn)의 벡터로 만들어 각 값에 따라 one-hot vector로 만들 수 있는 것이다. 

In [12]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

one-hot을 만들기 위해 모두 0인 벡터를 만든다. 
y 레이블을 보고 해당 위치에 1을 찍어서 one-hot vector를 채운다. 

In [13]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


꼭 위의 방식으로 다 할 필요는 없다. `torch.nn.functional` 라이브러리를 이용해 이를 단순화시킬 수 있다

In [ ]:
torch.log(F.softmax(z, dim = 1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [15]:
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [16]:
print(cost)

F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<MeanBackward0>)


tensor(1.4689, grad_fn=<NllLossBackward0>)

`(y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()` 로 힘들게 표현할 필요도 없다. 
여기에서 nll은 `negative log likelihood`의 약자이다. 

In [17]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

## Training with Low-level Cross Entropy Loss

In [19]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5], 
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]

y_train = [2, 2, 2, 1, 1, 1, 0, 0]

x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

|x_train| = (m, 4)
|y_train| = (m,)

4차원의 어떤 벡터를 받아서 어떤 class인지 예측할 수 있어야 한다. 
y = x를 one hot vector로 만들었을 때 1의 인덱스값

In [24]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000

for epoch in range(nb_epochs+1):
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d} / {} Cost : {:6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0 / 1000 Cost : 1.098612
Epoch  100 / 1000 Cost : 0.901535
Epoch  200 / 1000 Cost : 0.839114
Epoch  300 / 1000 Cost : 0.807826
Epoch  400 / 1000 Cost : 0.788472
Epoch  500 / 1000 Cost : 0.774822
Epoch  600 / 1000 Cost : 0.764449
Epoch  700 / 1000 Cost : 0.756191
Epoch  800 / 1000 Cost : 0.749398
Epoch  900 / 1000 Cost : 0.743671
Epoch 1000 / 1000 Cost : 0.738749


샘플수 : m, class : 3, dim : 4

현재 4 -> 3으로 가는 linear layer를 만든 상태

근데 이거보다 쉽게 만들어보자.

In [25]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000

for epoch in range(nb_epochs+1):
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d} / {} Cost : {:6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0 / 1000 Cost : 1.098612
Epoch  100 / 1000 Cost : 0.761050
Epoch  200 / 1000 Cost : 0.689991
Epoch  300 / 1000 Cost : 0.643229
Epoch  400 / 1000 Cost : 0.604117
Epoch  500 / 1000 Cost : 0.568255
Epoch  600 / 1000 Cost : 0.533922
Epoch  700 / 1000 Cost : 0.500291
Epoch  800 / 1000 Cost : 0.466908
Epoch  900 / 1000 Cost : 0.433507
Epoch 1000 / 1000 Cost : 0.399962


이렇게 위의 방식보다 간단하게 cross_entropy를 이용한 모델 학습을 할 수 있다. 
그런데, `nn.Module`를 이용하면 이것 보다 단순하게 할 수 있을까?

In [26]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # 위에서 말한대로 4 -> 3으로 바꾸기 때문에

    def forward(self, x):
        return self.linear(x)

In [28]:
model = SoftmaxClassifierModel()

In [29]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000

for epoch in range(nb_epochs+1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d} / {} Cost : {:6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0 / 1000 Cost : 1.845720
Epoch  100 / 1000 Cost : 0.647150
Epoch  200 / 1000 Cost : 0.568868
Epoch  300 / 1000 Cost : 0.515699
Epoch  400 / 1000 Cost : 0.471727
Epoch  500 / 1000 Cost : 0.432487
Epoch  600 / 1000 Cost : 0.395879
Epoch  700 / 1000 Cost : 0.360507
Epoch  800 / 1000 Cost : 0.325228
Epoch  900 / 1000 Cost : 0.289217
Epoch 1000 / 1000 Cost : 0.254086
